# Imports

In [ ]:
%matplotlib inline

import xarray as xr
import pandas as pd
from glob import glob
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import axes
import numpy as np
import numpy.matlib

import plotly.express as px

import seaborn as sns
# sns.set_theme(style="whitegrid")
# sns.set(rc={"figure.dpi":200,})
# sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
# sns.set_context('notebook')

import warnings
warnings.simplefilter('ignore')

from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from scipy import spatial

import random

# for regression P values
import statsmodels.api as sm
from sklearn_extra.cluster import KMedoids
from sklearn.feature_selection import chi2
from sklearn.metrics.pairwise import cosine_similarity

from scipy import stats
from scipy.cluster import hierarchy
from scipy.spatial import distance

In [ ]:
# pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(suppress=True)

In [ ]:
# For plotting maps
import os
os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "share", "proj")
from mpl_toolkits.basemap import Basemap

# Data Cleaning Functions

In [ ]:
def scale_values(arr):
    '''
    Scale an array between -1 to +1
    '''
    return 2.*(arr - np.min(arr))/np.ptp(arr)-1

In [ ]:
def get_cell_range(start, end,cell_width =10):
    '''
    get the ranges of the cells
    '''
    num_iter = (abs(start) + abs(end))/cell_width
    range_lst = []
    for i in range(int(num_iter)+1):
        range_lst.append(start+i*cell_width)
#         print(start+i*cell_width)
    return range_lst

In [ ]:
def get_year_month(df, yrmonth):
    df['time_counter'] = df['time_counter'].astype("string")
    df = df.loc[df['time_counter'].str.contains(yrmonth, case=False)]
    return df.reset_index()

In [ ]:
def round_nav_lat(df):
    '''
    Round up the coordinates to 2 decimal places
    '''
    df['nav_lat'] = df['nav_lat'].apply(lambda x:round(x,2))
    df['nav_lon'] = df['nav_lon'].apply(lambda x:round(x,2))
    return df

# Regression model

## Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def fit_random_forest_regression(grids_df_lst,feat_names,target):
    grid_reg_score = []
    grid_reg_coef = []
    
    data_count = []
    
    count = 0
    
    for grid_i in grids_df_lst:
        
        X = grid_i[feat_names].values
        y = np.array(grid_i[target].values)
   
        if np.isnan(X).any():
            print(X)
            raise ValueError
        
        if (len(X) == 0) or (len(y) == 0):
            grid_reg_score.append(None)
            grid_reg_coef.append(None)
        else:
        
            rf = RandomForestRegressor(n_estimators=150)
            rf.fit(X, y)

            grid_reg_score.append(rf.score(X, y))
            grid_reg_coef.append(rf.feature_importances_)
        data_count.append(len(X))
    
    
    
    df_column_names = ['cell_id','lon_min','lon_max','lat_min','lat_max', 'data_count',
                       'reg_score','reg_coef']        
    save_df = pd.DataFrame(columns=df_column_names)
    
    nav_lat_max_lst = []
    nav_lat_min_lst = []
    nav_lon_max_lst = []
    nav_lon_min_lst = []

    for grid_i in grids_df_lst:
        nav_lat_max_lst.append(grid_i['nav_lat'].max())
        nav_lat_min_lst.append(grid_i['nav_lat'].min())
        nav_lon_max_lst.append(grid_i['nav_lon'].max())
        nav_lon_min_lst.append(grid_i['nav_lon'].min())
    
    save_df['cell_id'] = range(0, len(grids_df_lst))
    save_df['lon_min'] = nav_lon_min_lst
    save_df['lon_max'] = nav_lon_max_lst
    save_df['lat_min'] = nav_lat_min_lst
    save_df['lat_max'] = nav_lat_max_lst
    save_df['data_count'] = data_count
    
    save_df['reg_score'] = grid_reg_score
    save_df['reg_coef'] = grid_reg_coef
    
    print("Returning values: ", df_column_names)
    return save_df


## MV Linear regression

In [ ]:

from scipy import stats
from sklearn.feature_selection import chi2


def fit_multivariate_lin_regression(grids_df_lst,feat_names,target, is_natural=True):
    '''
    https://satishgunjal.com/multivariate_lr_scikit/ 
    '''
    # Fit Regression model
    grid_reg_score = []
    grid_reg_coef = []
    grid_reg_intercept = []
    
    p_values_intercept = []
    p_values_sst = []
    p_values_dic = []
    p_values_alk = []
    
    data_count = []
    
    count = 0
    
    for grid_i in grids_df_lst:
#         feat_lst=[]
#         for index, row in grid_i.iterrows():
#             row_feat_lst = [row[feat] for feat in feat_names]
#             feat_lst.append(np.array(row_feat_lst))
        
#         X = grid_i.values[:,2:5]
        X = grid_i[feat_names].values
        y = np.array(grid_i[target].values)
   
        if np.isnan(X).any():
            print(X)
            raise ValueError
        
        if (len(X) == 0) or (len(y) == 0) or(len(X) == 1) or (len(y) == 1) :
            grid_reg_score.append(None)
            grid_reg_coef.append(None)
            grid_reg_intercept.append(None)
            p_values_intercept.append(None)
            p_values_sst.append(None)
            p_values_dic.append(None)
            p_values_alk.append(None)
            
            data_count.append(len(X))
            
        else:
            data_count.append(len(X))
        
            # y = mX + c
            lin_reg = linear_model.LinearRegression().fit(X, y)
#             lin_reg = linear_model.Ridge(alpha=1.0).fit(X, y)

            X2 = sm.add_constant(X)
            est = sm.OLS(y, X2).fit()
            _p_ = est.pvalues
            # print(f"P values are: {_p_}")
            if len(_p_) == 4:
                p_values_intercept.append(_p_[0])
                p_values_sst.append(_p_[1])
                p_values_dic.append(_p_[2])
                p_values_alk.append(_p_[3])
            else:
                p_values_intercept.append(None)
                p_values_sst.append(None)
                p_values_dic.append(None)
                p_values_alk.append(None)
            
            grid_reg_coef.append(lin_reg.coef_) #slope m
            grid_reg_intercept.append(lin_reg.intercept_) #intercept c
            grid_reg_score.append(lin_reg.score(X, y)) #quality or a confidence score
               
            count = count + 1
    
    
    df_column_names = ['cell_id','lon_min','lon_max','lat_min','lat_max', 'data_count',
                       'reg_score','reg_coef', 'reg_intercept','p_intercept']
    
    for fn in feat_names:
        df_column_names.append(f"p_{fn}")
        
    print("Returning values: ", df_column_names)
        
    save_df = pd.DataFrame(columns=df_column_names)
    nav_lat_max_lst = []
    nav_lat_min_lst = []
    nav_lon_max_lst = []
    nav_lon_min_lst = []

    for grid_i in grids_df_lst:
        nav_lat_max_lst.append(grid_i['nav_lat'].max())
        nav_lat_min_lst.append(grid_i['nav_lat'].min())
        nav_lon_max_lst.append(grid_i['nav_lon'].max())
        nav_lon_min_lst.append(grid_i['nav_lon'].min())
    
    save_df['cell_id'] = range(0, len(grids_df_lst))
    save_df['lon_min'] = nav_lon_min_lst
    save_df['lon_max'] = nav_lon_max_lst
    save_df['lat_min'] = nav_lat_min_lst
    save_df['lat_max'] = nav_lat_max_lst
    save_df['data_count'] = data_count
    
    save_df['reg_score'] = grid_reg_score
    save_df['reg_coef'] = grid_reg_coef
    save_df['reg_intercept'] = grid_reg_intercept
    
    save_df['p_intercept'] = p_values_intercept
    save_df['p_sst'] = p_values_sst
    if is_natural:
        save_df['p_dicp'] = p_values_dic
    else:
        save_df['p_dic'] = p_values_dic
    save_df['p_alk'] = p_values_alk
        
    return save_df


In [ ]:
def save_results(feat_names,target,grids_df_lst,grid_reg_score, grid_reg_coef, grid_reg_intercept,month,cell_width):
    save_df = pd.DataFrame(columns=['cell_id','lon_min','lon_max','lat_min','lat_max',
                                'reg_score','reg_coef', 'reg_intercept'])
    nav_lat_max_lst = []
    nav_lat_min_lst = []
    nav_lon_max_lst = []
    nav_lon_min_lst = []

    for grid_i in grids_df_lst:
        nav_lat_max_lst.append(grid_i['nav_lat'].max())
        nav_lat_min_lst.append(grid_i['nav_lat'].min())
        nav_lon_max_lst.append(grid_i['nav_lon'].max())
        nav_lon_min_lst.append(grid_i['nav_lon'].min())
    
    save_df['cell_id'] = range(0, len(grids_df_lst))
    save_df['lon_min'] = nav_lon_min_lst
    save_df['lon_max'] = nav_lon_max_lst
    save_df['lat_min'] = nav_lat_min_lst
    save_df['lat_max'] = nav_lat_max_lst
    save_df['reg_score'] = grid_reg_score
    save_df['reg_coef'] = grid_reg_coef
    save_df['reg_intercept'] = grid_reg_intercept
    
    feat = '_'.join(feat_names)
    
    filepath = f"../csv_files/reg_model_{month}_{cell_width}_{target}_{feat}.csv"
    save_df.to_csv(filepath)
    print(f"\n File saved at {filepath}")
    return save_df

In [ ]:
def build_grids(df_month,cell_width=2):
    # Prepare the cells
    nav_lat_grids = get_cell_range(start = -90, end = 90 ,cell_width = cell_width)
    nav_lon_grids = get_cell_range(start = -180, end = 180 ,cell_width = cell_width)
    
    if nav_lat_grids[-1] != 90:
        nav_lat_grids.append(90)
        
    if nav_lon_grids[-1] != 180:
        nav_lon_grids.append(180)
        
    # Build the grids. Store in a list.
    grids_df_lst=[]
    for lat_i in range(len(nav_lat_grids)):
        for lon_j in range(len(nav_lon_grids)):
            if((nav_lat_grids[lat_i] == 90) or (nav_lon_grids[lon_j] == 180)):
                break
            elif ((lat_i == len(nav_lat_grids) - 1) or (lon_j == len(nav_lon_grids) - 1)):
                break
            else:
                _df_ = df_month.loc[
                    (df_month['nav_lat'] >= nav_lat_grids[lat_i]) & 
                    (df_month['nav_lat'] <  nav_lat_grids[lat_i+1]) &
                    (df_month['nav_lon'] >= nav_lon_grids[lon_j]) & 
                    (df_month['nav_lon'] <  nav_lon_grids[lon_j+1])
                                ]
                grids_df_lst.append(_df_)
    
    print(f"\n Total no. of generated cells: {len(grids_df_lst)}")
    
    return grids_df_lst

In [ ]:
def run_model(model_df, feat_names, target, cell_width, is_natural):
    # build grid cells
    grids_df_lst = build_grids(model_df,cell_width)
    # fit linear regression
    save_df = fit_multivariate_lin_regression(grids_df_lst,feat_names,target, is_natural)   
    return save_df, grids_df_lst

# Run MVLR in 2x2 boxes

In [ ]:
months_12 = {
            'jan':'-01-',
            'feb':'-02-',
            'mar':'-03-',
            'apr':'-04-',
            'may':'-05-',
            'jun':'-06-',
            'jul':'-07-',
            'aug':'-08-',
            'sep':'-09-',
            'oct':'-10-',
            'nov':'-11-',
            'dec':'-12-',
}

In [ ]:
%%time

# _month = 'jan'

months = list(months_12.keys())
cell_width = 2

## function to set up dataframe for Hierarchical clustering
fn_sst = lambda row: row['reg_coef'][0]
fn_dic = lambda row: row['reg_coef'][1]
fn_alk = lambda row: row['reg_coef'][2]

for yr in range(1958, 1980):
    for mnth in months_12:

        print(yr, mnth)
        data_df = pd.read_pickle(f"../carbon_data_preprocessed/ocean_data_{yr}_df.pkl")
    
        if 'sosstsst' in data_df.columns:
            data_df = data_df.rename(columns={'sosstsst': 'SST'})
        
        ## Remove known outliers
        data_df = data_df.loc[data_df['DICP'] >= 1500]
        # data_df = data_df.loc[data_df['DIC'] >= 1500]
        # Select ALK above 1700
        data_df = data_df.loc[data_df['ALK'] >= 1700]
        # Select SST below 40
        data_df = data_df.loc[data_df['SST'] <= 40]
        # Select fco2 below 500
        data_df = data_df.loc[data_df['fco2_pre'] <= 500]
    
        data_df = round_nav_lat(data_df)
        df_month = get_year_month(df = data_df , yrmonth = months_12[mnth])
        df_month['area'] = df_month['e1t'] * df_month['e2t']
    
        print(df_month.columns)
    
        print("Gridding 2x2")
        grids_df_lst = build_grids(df_month,cell_width=cell_width)
    
        print("Running MVLR")
        reg_df_mvlr = fit_multivariate_lin_regression(grids_df_lst = grids_df_lst ,
                                        feat_names=['SST','DICP', 'ALK'],target='fco2_pre')
    
        ## Remove grids with Zero number of coordinates
        reg_df_mvlr = reg_df_mvlr[reg_df_mvlr.data_count != 0]
        reg_df_mvlr = reg_df_mvlr[reg_df_mvlr.data_count != 1]
        
        ## Regression above 96% of significance
        reg_df_p = reg_df_mvlr[(reg_df_mvlr.p_sst <= 0.04) & (reg_df_mvlr.p_dicp <= 0.04)& (reg_df_mvlr.p_alk <= 0.04)]
    
        ## Saving the slopes
        hc_df = pd.DataFrame()
        hc_df['cell_id'] = reg_df_p['cell_id']
        hc_df['slope_sst'] = reg_df_p.apply(fn_sst,axis=1)
        hc_df['slope_dicp'] = reg_df_p.apply(fn_dic,axis=1)
        hc_df['slope_alk'] = reg_df_p.apply(fn_alk,axis=1)
    
        hc_df['slope_sst_std'] = (hc_df['slope_sst']-hc_df['slope_sst'].mean())/hc_df['slope_sst'].std()
        hc_df['slope_dicp_std'] = (hc_df['slope_dicp']-hc_df['slope_dicp'].mean())/hc_df['slope_dicp'].std()
        hc_df['slope_alk_std'] = (hc_df['slope_alk']-hc_df['slope_alk'].mean())/hc_df['slope_alk'].std()
    
        appended_data = []
        for index, row in hc_df.iterrows():
            # get the corresponding grid from the list of grids
            _df_ = grids_df_lst[index]
            _df_['grid_id'] = index
            _df_['slope_sst'] = row['slope_sst']
            _df_['slope_dicp'] = row['slope_dicp']
            _df_['slope_alk'] = row['slope_alk']
            _df_['slope_sst_std'] = row['slope_sst_std']
            _df_['slope_dicp_std'] = row['slope_dicp_std']
            _df_['slope_alk_std'] = row['slope_alk_std']
            appended_data.append(_df_)
            
        appended_data = pd.concat(appended_data)
        appended_data.to_pickle(f"output_files/spatial_regression_{yr}_{mnth}.pkl")
        print()

# Biome detection- Jan, 2009

In [ ]:
_year = 2009
_month = 'jan'

In [ ]:
data_df = pd.read_pickle(f"output_files/spatial_regression_{_year}_{_month}.pkl")
data_df

# Adaptive HC

In [ ]:
hc_df = data_df.groupby('cell_id').mean()

# Dendrogram

In [ ]:
X = hc_df[['slope_sst_std', 'slope_dicp_std', 'slope_alk_std']].values
Z = hierarchy.linkage(X, method='ward')
Z

In [ ]:
Z.shape

In [ ]:
# hierarchy.fcluster(Z, t=100, criterion='distance')
# hierarchy.dendrogram(Z, color_threshold=0)
# plt.show()

## Automtic Cut Function

In [ ]:
tot_children = len(norm_hc_df)
tot_children

In [ ]:
def get_feat_arr(dendro_dict_lst, index_, arr):
    feat_stack = np.array([])
    for item in dendro_dict_lst:
            if item['id'] == index_:
                feat_stack = item['feat_stack']
                break
    if feat_stack.size == 0:
        feat_stack = [arr[index_]]
    return feat_stack

In [ ]:
def get_grid_index_list(dendro_dict_lst, index_, df):
    _l_ = []
    for item in dendro_dict_lst:
            if item['id'] == index_:
                _l_ = item['grid_index']
                break
    if len(_l_) == 0:
        _l_ = [df.index[index_]]
    return _l_

In [ ]:
# https://stackoverflow.com/questions/73103010/matching-up-the-output-of-scipy-linkage-and-dendrogram

def append_index(n, i, cluster_id_list):
    # refer to the recursive progress in
    # https://github.com/scipy/scipy/blob/4cf21e753cf937d1c6c2d2a0e372fbc1dbbeea81/scipy/cluster/hierarchy.py#L3549

    # i is the idx of cluster(counting in all 2 * n - 1 clusters)
    # so i-n is the idx in the "Z"
    if i < n:
        return
    aa = int(Z[i - n, 0])
    ab = int(Z[i - n, 1])

    append_index(n, aa, cluster_id_list)
    append_index(n, ab, cluster_id_list)

    cluster_id_list.append(i-n)
    # Imitate the progress in hierarchy.dendrogram
    # so how `i-n` is appended , is the same as how the element in 'icoord'&'dcoord' be.
    return

def get_linkid_clusterid_relation(Z):
    Zs = Z.shape
    n = Zs[0] + 1
    i = 2 * n - 2
    cluster_id_list = []
    append_index(n, i, cluster_id_list)
    # cluster_id_list[i] is the cluster idx(in Z) that the R['icoord'][i]/R['dcoord'][i] corresponds to

    dict_linkid_2_clusterid = {linkid: clusterid for linkid, clusterid in enumerate(cluster_id_list)}
    dict_clusterid_2_linkid = {clusterid: linkid for linkid, clusterid in enumerate(cluster_id_list)}
    return dict_linkid_2_clusterid, dict_clusterid_2_linkid

dict_linkid_2_clusterid, dict_clusterid_2_linkid = get_linkid_clusterid_relation(Z)

## Build Dendro dict

In [ ]:
%%time
clust_num = tot_children
print(f'Start Cluster number = {clust_num}')
dendro_dict_lst = []
# index_count = len(Z) - 1
index_count = 0
for row in Z:
    dendro_dict ={}
    dendro_dict['id'] = clust_num    
    dendro_dict['info'] = row
    dendro_dict['feat_stack'] = np.array([])
    dendro_dict['variance'] = np.array([])
    dendro_dict['Z_index'] = index_count
    dendro_dict['grid_index'] = []

    dendro_dict_lst.append(dendro_dict)
    clust_num = clust_num + 1    
    index_count = index_count + 1

print(f'End Cluster number = {clust_num}')

In [ ]:
%%time

count = 0
for dendro in dendro_dict_lst:
    arr = dendro['info']
    first_index = int(arr[0])
    second_index = int(arr[1])
    child_num = arr[3]
    if child_num == 2:
        dendro['feat_stack'] =np.append([X[first_index]],[X[second_index]],axis = 0)
        dendro['variance'] = np.var(dendro['feat_stack'])
        _l_ = [norm_hc_df.index[first_index]]
        _l_.append(norm_hc_df.index[second_index])
        dendro['grid_index'] = _l_

        count = count + 1

print(count)

In [ ]:
%%time

count = 0

for dendro in dendro_dict_lst:
    arr_ = dendro['info']
    first_index = int(arr_[0])
    second_index = int(arr_[1])
    child_num = arr_[3]
    if child_num > 2:
#         print(arr[3])
#         print(first_index)
#         print(second_index)
        left_index = get_feat_arr(dendro_dict_lst = dendro_dict_lst, 
                         index_ = first_index, 
                         arr = X)
        
        right_index = get_feat_arr(dendro_dict_lst = dendro_dict_lst, 
                         index_ = second_index, 
                         arr = X)  
        dendro['feat_stack'] =np.append(left_index,right_index,axis = 0)
        dendro['variance'] = np.var(dendro['feat_stack'])
        
        left_grid_index = get_grid_index_list(dendro_dict_lst = dendro_dict_lst,
                                              index_ = first_index, df = norm_hc_df)
        right_grid_index = get_grid_index_list(dendro_dict_lst = dendro_dict_lst,
                                               index_ = second_index, df = norm_hc_df)
        # appending lists to one list
        dendro['grid_index'] = [*left_grid_index, *right_grid_index]


        count = count + 1    
#         break
# print(dendro)
print(count)    

## Run over multiple set of del_var and del_dist

In [ ]:
def run_through_dendrogram(delta_var, delta_dist, dendro_dict_lst):

    process_left = []
    process_right = []

    cluster_list = []
    z_index_list = []

    # get the top most element from the dendrogram and set is as "Start".
    start = dendro_dict_lst[-1]

    head = start['id']
    current_left = start['info'][0]
    current_right = start['info'][1]
    current_height = start['info'][2]
    current_variance = start['variance']
    current_z_index = start['Z_index']

    while(head):
        left_height = None
        left_variance = None
        left_z_index = None
        right_height = None
        right_variance = None
        right_z_index = None

        # left_child_num = 1
        # right_child_num = 1

        for item in dendro_dict_lst:
            if item['id'] == current_left:
                left_height = item['info'][2]
                left_variance = item['variance']
                left_z_index = item['Z_index']

        for item in dendro_dict_lst:
            if item['id'] == current_right:
                right_height = item['info'][2]
                right_variance = item['variance']
                right_z_index = item['Z_index']

        if right_height:
            change_height_right = abs(current_height - right_height)
            change_variance_right = abs(current_variance - right_variance)
            if ((change_height_right > delta_dist) and (change_variance_right > delta_var)):
                process_right.append(current_right)
            else:
                cluster_list.append(current_right)
                z_index_list.append(right_z_index)

        else:
            cluster_list.append(current_right)
    #         z_index_list.append(current_right)

        if left_height:
            change_height_left = abs(current_height - left_height)
            change_variance_left = abs(current_variance - left_variance)

            if ((change_height_left > delta_dist) and (change_variance_left > delta_var)): 
                #change_variance_left = np.inf
                    process_left.append(current_left)
            else:
                    # splitting the current head
                    cluster_list.append(current_left)
                    z_index_list.append(left_z_index)
        else:
            cluster_list.append(current_left)
    #         z_index_list.append(current_left)

        ## Reassign head
        if process_left:
            head = process_left.pop()
        elif process_right:
            head = process_right.pop()
        else:
            head = None

        ## Reassign
        if head:
            for item in dendro_dict_lst:
                if item['id'] == head:
                    current_left = item['info'][0]
                    current_right = item['info'][1]
                    current_height = item['info'][2]
                    current_variance = item['variance']
                    current_z_index = item['Z_index']
                    
    return cluster_list

In [ ]:
def get_all_clusters(cluster_list):
    heights_lst = []
    grid_index_list = []
    for c in cluster_list:
        found = False
        for item in dendro_dict_lst:
                if item['id'] == c:
    #                 print(item['id'])
                    found = True
                    ht = item['info'][2]
                    heights_lst.append(ht)
                    grid_index_list.append(item['grid_index'])
        if not found:
#             print(f"!!! Singleton cluster at {c}")
            heights_lst.append(0.0)
            grid_index_list.append([norm_hc_df.index[int(c)]])
    
    ## Extract clusters
    cluster_lbl = 1
    l__indices = []
    l__lbls = []
    for grid_list in grid_index_list:
        if type(grid_list) != list:
            l__indices.append(grid_list)
            l__lbls.append(cluster_lbl)
        else:
            for i in grid_list:
                # print(f'going wrong: ', i)
                l__indices.append(i)
                l__lbls.append(cluster_lbl)
        cluster_lbl = cluster_lbl + 1

    _df_ = pd.DataFrame(l__lbls, index =l__indices,columns =['cluster'])
    cluster_df = pd.merge(norm_hc_df, _df_, left_index=True, right_index=True)
    return cluster_df

In [ ]:
def get_bic_aic_score(cluster_df):
    #size of data set -> N datapoints with d no. of features
    N, d = X.shape
    #unique labels
    labels_unique = cluster_df['cluster'].unique()

    loglikelihood = 0  
    for lbl in labels_unique:
        df_lbl = cluster_df.loc[cluster_df['cluster'] == lbl]
        loglikelihood =loglikelihood + np.log(len(df_lbl)/N)

    #BIC
    BIC = -2 * loglikelihood + d * np.log(N)
    AIC = -2 * loglikelihood + 2*d
    return BIC, AIC, labels_unique

### Set distance variance list

In [ ]:
del_variance_list = [0.1,0.5, 0.8, 1.0]
del_distance_list = [20,25,28,30,]

In [ ]:
%%time

plot_del_var = []
plot_del_dist = []
bic_scores = []
aic_scores = []
tot_clusters = []

for del_dist in del_distance_list:
    for del_var in del_variance_list:
#         print(del_var, del_dist)
        cluster_list = run_through_dendrogram(del_var, del_dist, dendro_dict_lst)
        cluster_df = get_all_clusters(cluster_list)
        BIC, AIC, labels_unique = get_bic_aic_score(cluster_df)
        
        # Add to the lists
        plot_del_var.append(del_var)
        plot_del_dist.append(del_dist)
        bic_scores.append(BIC)
        aic_scores.append(AIC)
        tot_clusters.append(len(labels_unique))
        
#         if ((del_var == 0.5) and (del_dist == 15.0)):
#             print('hello', len(labels_unique))

In [ ]:
df = pd.DataFrame(list(zip(plot_del_dist, plot_del_var, bic_scores, tot_clusters )),
               columns =['Del_Dist', 'Del_Var', 'BIC', 'Clusters'])
df = df.round(2)
df

In [ ]:
# df.to_latex(index=False)

In [ ]:
df_pivot = df.pivot('Del_Dist', 'Del_Var', 'BIC')
df_pivot

In [ ]:
sns.heatmap(df_pivot, annot=True, fmt=".2f", cmap="summer_r", vmin=40, vmax=80, annot_kws={"fontsize":12})
plt.xlabel(" ")
plt.ylabel(" ")

In [ ]:
df_pivot = df.pivot('Del_Dist', 'Del_Var', 'Clusters')
df_pivot

In [ ]:
sns.heatmap(df_pivot, annot=True, cmap="summer_r", vmin=3, vmax=10, annot_kws={"fontsize":12})
plt.xlabel(" ")
plt.ylabel(" ")

### BIC Plots

In [ ]:
df_ = df.groupby(['Del_Dist'], as_index=False)['BIC', 'Clusters'].mean()
df_ = df_.astype({"Clusters":'int'})
df_

In [ ]:
t = df_['Del_Dist'].values
data1 = df_['BIC'].values
data2 = df_['Clusters']

# data2 = np.array(data2)


fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Change in distance', fontsize=15)
ax1.set_ylabel('Mean BIC', color=color, fontsize=15)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:brown'
ax2.set_ylabel('Mean #Cluster', color=color, fontsize=15)  # we already handled the x-label with ax1
ax2.scatter(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)
# ax2.set_major_locator(MaxNLocator(integer=True))

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
df_ = df.groupby(['Del_Var'], as_index=False)['BIC', 'Clusters'].mean()

t = df_['Del_Var'].values
data1 = df_['BIC'].values
data2 = df_['Clusters'].values

In [ ]:
data2 = np.array(data2).astype(int).tolist()

print(data2)

fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Change in variance', fontsize=15)
ax1.set_ylabel('Mean BIC', color=color, fontsize=15)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:brown'
ax2.set_ylabel('Mean #Cluster', color=color, fontsize=15)  # we already handled the x-label with ax1
ax2.scatter(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# bic_cluster_num_fig = px.scatter(df, x="Del_Dist", y="Del_Var",size='Clusters',
#                                 color= 'BIC', hover_data=['Clusters', 'BIC'],)
# bic_cluster_num_fig.update_layout(title = 'BIC Score and total no. of clusters generated wrt change in variance and distance',)
    

# Distance - Variance Selection

In [ ]:
%%time


## For Jan 2009
delta_dist = 25.0
delta_var = 0.1

process_left = []
process_right = []

cluster_list = []
z_index_list = []

# get the top most element from the dendrogram and set is as "Start".
start = dendro_dict_lst[-1]

head = start['id']
current_left = start['info'][0]
current_right = start['info'][1]
current_height = start['info'][2]
current_variance = start['variance']
current_z_index = start['Z_index']

test_count = 0

count_stop = 0

while(head):
    print()
    print('New head.')
    left_height = None
    left_variance = None
    left_z_index = None
    right_height = None
    right_variance = None
    right_z_index = None
    
    # left_child_num = 1
    # right_child_num = 1

    for item in dendro_dict_lst:
        if item['id'] == current_left:
            left_height = item['info'][2]
            left_variance = item['variance']
            left_z_index = item['Z_index']

    for item in dendro_dict_lst:
        if item['id'] == current_right:
            right_height = item['info'][2]
            right_variance = item['variance']
            right_z_index = item['Z_index']
        

    if left_height:
        change_height_left = abs(current_height - left_height)
        change_variance_left = abs(current_variance - left_variance)
        
        print("Change in height:", change_height_left)
        print("Change in variance:", change_variance_left)

        if ((change_height_left > delta_dist) and (change_variance_left > delta_var)): 
            #change_variance_left = np.inf
            print('went down.')
            process_left.append(current_left)
        else:
            
            # splitting the current head
            cluster_list.append(current_left)
            z_index_list.append(left_z_index)
            count_stop = count_stop + 1
            print(f'stopped. - {count_stop}')
            
    else:
        cluster_list.append(current_left)
        z_index_list.append(current_left)
        count_stop = count_stop + 1
        print(f'stopped. - {count_stop}')
        
    if right_height:
        change_height_right = abs(current_height - right_height)
        change_variance_right = abs(current_variance - right_variance)
        
        print("Change in height:", change_height_right)
        print("Change in variance:", change_variance_right)

        if ((change_height_right > delta_dist) and (change_variance_right > delta_var)):
            print('went down.')
            process_right.append(current_right)
        else:
            cluster_list.append(current_right)
            z_index_list.append(right_z_index)
            count_stop = count_stop + 1
            print(f'stopped. - {count_stop}')
            

    else:
        cluster_list.append(current_right)
        z_index_list.append(current_right)
        count_stop = count_stop + 1
        print(f'stopped. - {count_stop}')
        
    ## Reassign head
    if process_left:
        head = process_left.pop()
    elif process_right:
        head = process_right.pop()
    else:
        head = None
    
    ## Reassign
    if head:
        for item in dendro_dict_lst:
            if item['id'] == head:
                current_left = item['info'][0]
                current_right = item['info'][1]
                current_height = item['info'][2]
                current_variance = item['variance']
                current_z_index = item['Z_index']


        
#     test_count = test_count + 1
#     if test_count == 100:
#         break
cluster_list

In [ ]:
heights_lst = []
grid_index_list = []
for c in cluster_list:
    found = False
    for item in dendro_dict_lst:
            if item['id'] == int(c):
#                 print(item['id'])
                found = True
                ht = item['info'][2]
                heights_lst.append(ht)
                grid_index_list.append(item['grid_index'])
    if not found:
        print("singleton found")
        print(c)
        heights_lst.append(0.0)
        grid_index_list.append([norm_hc_df.index[int(c)]])
len(grid_index_list)

In [ ]:
len(z_index_list)

In [ ]:
len(grid_index_list)

# Plot Dendrogram

In [ ]:
%%time
plt.figure(figsize=(10, 8), dpi=300).set_facecolor('white')
# plt.title(f"Dendrograms at delta_var = {round(delta_var,2)}, delta_dist = {round(delta_dist,2)}, #Clusters = {len(heights_lst)}",
#           fontsize=16)

dend = hierarchy.dendrogram(Z, color_threshold=0, above_threshold_color='k')
LBL_COUNT = 1

for z, h in zip(z_index_list, heights_lst):
    link_id = dict_clusterid_2_linkid[z]
    i = dend['icoord'][link_id]
    x = 0.5 * sum(i[1:3])
    y = h
    plt.plot(x, y, 'ro',c='red', markersize=8) #label = f'height = {y}'
#     plt.plot(x, y, 'ro',c=cluster_color_dict[LBL_COUNT], markersize=8) #label = f'height = {y}'
    
#     label = regime_names_dict[LBL_COUNT]
    plt.annotate(LBL_COUNT, # this is the text
                (x,y), # these are the coordinates to position the text
                textcoords="offset points", # how to position the text
                xytext=(5,5), # distance from text to points (x,y)
                ha='left', # horizontal alignment can be left, right or center
                color='red',
#                 color=cluster_color_dict[LBL_COUNT],
                fontsize=18, weight='bold',)
#     plt.axhline(y=h, color='black', linestyle='--', linewidth = 0.5, label = f'{LBL_COUNT} -> {h}')  
    
    LBL_COUNT = LBL_COUNT + 1
    
# # for height in heights_lst:
# #     plt.axhline(y=height, color='black', linestyle='--', label = f'{height}')  
# # plt.legend()

plt.xlabel("Individual grid cells containing RCs of SST, DIC, and ALK", fontsize=16)
plt.ylabel("Euclidean Distance", fontsize =16)
plt.xticks([], []) # Remove X axis ticks / grid cell ids
import matplotlib as mpl
# mpl.rcParams['xtick.labelsize'] = 16 
mpl.rcParams['ytick.labelsize'] = 16
plt.savefig(f"output_tracking/Dendrogram_{_year}_{_month}_{len(heights_lst)}_clusters",
            dpi=300, bbox_inches='tight')
plt.show()

## Extract Clusters

In [ ]:
cluster_list

In [ ]:
regime_names_dict = {
    1: 'ICE I',
    2: 'ICE II',
    3: 'SUBTR I',
    4: 'SUBTR II',
    6: 'SUBP + UP I',
    7: 'SUBP + UP II',
    5: 'SUBP + UP III',
}
len(regime_names_dict)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define the cluster_color_dict
cluster_color_dict = {
    1: sns.color_palette("light:blue", as_cmap=False, n_colors=10)[3],
    2: 'blue',
    3: sns.color_palette("autumn", as_cmap=False, n_colors=5)[4],#'yellow',
    4: 'orange',
    6: 'gray',
    7: sns.color_palette("light:green", as_cmap=False, n_colors=5)[2],
    5: 'green', 
}

# Define the figure and axis
fig, ax = plt.subplots(figsize=(8, 2), dpi=1200)

# Set the limits of the x and y axes
ax.set_xlim(0, len(cluster_color_dict) * 2)
ax.set_ylim(0, 2)

# Iterate over the cluster_color_dict to plot the colored boxes
for i, (cluster, color) in enumerate(cluster_color_dict.items()):
    # Plot the colored box
    ax.add_patch(plt.Rectangle((i * 1.1,1), 1, 0.5, color=color)) # First 1 is for padding between the colors
    # Plot the number underneath the box
#     ax.text(i * 2 + 0.5, -0.5, str(cluster), ha='center', va='center')

# Hide the axes
ax.axis('off')

# Show the plot
plt.show()


In [ ]:
cluster_lbl = 1
# cluster_lbl = 0
l__indices = []
l__lbls = []
for grid_list in grid_index_list:
    if type(grid_list) != list:
        l__indices.append(grid_list)
        l__lbls.append(cluster_lbl)
    else:
        for i in grid_list:
            # print(f'going wrong: ', i)
            l__indices.append(i)
            l__lbls.append(cluster_lbl)
    cluster_lbl = cluster_lbl + 1

In [ ]:
_df_ = pd.DataFrame(l__lbls, index =l__indices,columns =['cluster'])
hc_df = pd.merge(norm_hc_df, _df_, left_index=True, right_index=True)
hc_df

## BIC Scores

In [ ]:
labels_all = hc_df['cluster'].values #all labels
#size of data set -> N datapoints with d no. of features
N, d = X.shape
#unique labels
labels_unique = hc_df['cluster'].unique()

loglikelihood = 0  
for lbl in labels_unique:
    _df_ = hc_df.loc[hc_df['cluster'] == lbl]
    loglikelihood =loglikelihood + np.log(len(_df_)/N)

#BIC
BIC = -2 * loglikelihood + d * np.log(N)
AIC = -2 * loglikelihood + 2*d
b_score = BIC
b_score

## Save clusters

In [ ]:
appended_data = []
for index, row in hc_df.iterrows():
    # get the corresponding grid from the list of grids
    _df_ = grids_df_lst[index]
    _df_['cell_id'] = index
    _df_['cluster'] = row['cluster']
    _df_['slope_sst'] = row['slope_sst']
    _df_['slope_sst_std'] = row['slope_sst_std']
    if is_natural:
        _df_['slope_dicp'] = row['slope_dicp']
        _df_['slope_dicp_std'] = row['slope_dicp_std']
    else:
        _df_['slope_dic'] = row['slope_dic']
#         _df_['slope_dic_std'] = row['slope_dic_std']
        
    _df_['slope_alk'] = row['slope_alk']
    _df_['slope_alk_std'] = row['slope_alk_std']
#     _df_['slope_sal'] = row['slope_sal']
#     _df_['slope_sal_std'] = row['slope_sal_std']
    _df_['BIC'] = BIC
    _df_['delta_var'] = delta_var
    _df_['delta_dist'] = delta_dist
    appended_data.append(_df_)

In [ ]:
appended_data = pd.concat(appended_data)
appended_data

## Project on Map

In [ ]:

plt.figure(figsize=(14, 10), edgecolor='w')
regime_map = Basemap(projection='cyl', resolution='c',
            llcrnrlat=-90, urcrnrlat=90,
            llcrnrlon=-180, urcrnrlon=180, )
## Draw the coast.
# regime_map.drawcoastlines()

## Draw the land shades.
# regime_map.shadedrelief()

## Fill the land mass and lakes
regime_map.fillcontinents(color='black') #color_lake='aqua'

## draw parallels and meridians.
# regime_map.drawparallels(np.arange(-90,91,10),labels=[1,1,0,1], fontsize=12)
# regime_map.drawmeridians(np.arange(-180,181,10),labels=[1,1,0,1], rotation=45, fontsize=12)

##color the sea/oceans
# regime_map.drawmapboundary(fill_color='aqua')

count = 0
arr = np.sort((appended_data['cluster'].unique()))

for cluster_num in arr:
    _df_ = appended_data.loc[appended_data['cluster'] == cluster_num]
    lbl = int(cluster_num)
#     lbl = f"{int(cluster_num)} --> {len(_df_)} ({round(_df_['area'].sum()/1000000,2)} km sq)"
    regime_map.scatter(_df_['nav_lon'], _df_['nav_lat'], 
                       latlon=True,marker='.', 
#                        color=cluster_color_dict[cluster_num],
                       color=clrs_original[count],
                       label = lbl,
                       s=5)
    count = count + 1

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
# plt.legend(by_label.values(), by_label.keys())
lgnd = plt.legend(by_label.values(), by_label.keys(),loc='lower center', ncol=8, fontsize=17,
                  bbox_to_anchor=(0.5, -0.2))
#                   bbox_to_anchor=(1.0, 1.0))
for handle in lgnd.legendHandles:
    handle.set_linewidth(12)

plt.title(f'Carbon Regimes of {_year}, {_month} with delta_dist = {delta_dist}, delta_var={delta_var}',fontsize=20)

plt.show()

In [ ]:
appended_data.to_pickle(f"output_files/clusters_spatial_regression_2009_jan.pkl")

# Analysis for Jan 2009

In [ ]:
hc_df.groupby("cluster")['slope_sst_std', 'slope_dicp_std', 'slope_alk_std'].median()

In [ ]:
regime_mean = hc_df.groupby("cluster")['slope_sst_std', 'slope_dicp_std', 'slope_alk_std'].mean().reset_index()
regime_mean.round(3)

In [ ]:
regime_mean = hc_df.groupby("cluster")['slope_sst_std', 'slope_dicp_std', 'slope_alk_std'].mean().reset_index()
regime_mean.round(3)

In [ ]:
regime_var = hc_df.groupby("cluster")['slope_sst_std', 'slope_dicp_std', 'slope_alk_std'].var().reset_index()
regime_var.round(3)

In [ ]:
regime_mean['slope_sst_std']

In [ ]:
# regime_names = ['ICE II', 'ICE I', 'SUBTR II', 'SUBTR I', 'SUBP + UP II', 'ICE III', 'SUBP + UP I']
regime_names_dict = {
    1: 'ICE I',
    2: 'ICE II',
    3: 'SUBTR I',
    4: 'SUBTR II',
    6: 'SUBP + UP I',
    7: 'SUBP + UP II',
    5: 'SUBP + UP III',
}
len(regime_names_dict)

In [ ]:
def order_column_values_by_regime_name(regime_mean, col_name):
    new_order = []
    new_cluster_order = []
    new_keys = list(regime_names_dict.keys())
    new_keys.reverse()
    for key in new_keys:
        new_order.append(regime_mean.loc[regime_mean['cluster'] == key][col_name].values[0])
        new_cluster_order.append(key)
    return new_order, new_cluster_order

In [ ]:
order_column_values_by_regime_name(regime_mean, 'slope_alk_std')

In [ ]:
# Plot grouped horizontal bar chart for each cluster
fig, ax = plt.subplots(figsize=(10, 6),) #dpi=1200
# ax.grid(color='grey', alpha=0.2)
ax.grid(zorder=0)

# Get number of clusters
num_clusters = len(regime_mean)

# Set width for each bar group
bar_width = 0.2

# Set positions for each cluster starting from 1
positions = np.arange(1, num_clusters + 1)

# Plot bars for each column
for i, col in enumerate(regime_mean.columns[1:]):  # Exclude "Cluster ID" column
    if col == 'slope_dicp_std':
        lbl = 'DIC'
    else:
        lbl = col.split("_")[1].upper()
    # Determine position for the bar group
    pos = positions + i * bar_width - bar_width * (len(regime_mean.columns[1:]) - 1) / 2
#     print(col)
#     print(pos)
#     print()
    
    # Plot bars
    values, new_cluster_order = order_column_values_by_regime_name(regime_mean, col)
    if col == 'slope_sst_std':
        ax.barh(pos, values, bar_width, label=lbl, fill=False, hatch='|', zorder=3)
    if col == 'slope_dicp_std':
        ax.barh(pos, values, bar_width, label=lbl, fill=False, hatch='..', zorder=3)
    if col == 'slope_alk_std':
        ax.barh(pos, values, bar_width, label=lbl, fill=False, hatch='///', zorder=3)


# Add labels, legend, and title
ax.set_yticks(positions)
## Update the cluster labels
# ax.set_yticklabels(regime_mean.cluster)
y_labels = list(regime_names_dict.values())
y_labels.reverse()
ax.set_yticklabels(y_labels, weight='bold')

for ytick, cluster_num in zip(ax.get_yticklabels(), new_cluster_order):
    ytick.set_color(cluster_color_dict[cluster_num])

legend_properties = {'weight':'bold', 'size':16}
ax.legend(loc='lower right', prop=legend_properties)

ax.set_xlabel('Mean normalized RCs', fontsize=24)
ax.set_ylabel('Carbon regimes', fontsize=24)
# ax.set_title('Mean regression coeffcients of drivers for each carbon provinces', fontsize=15)

import matplotlib as mpl
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20

# Show plot
plt.savefig(f"output_tracking/figs/cluster_details_2009_jan",
            dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Plot grouped horizontal bar chart for each cluster
fig, ax = plt.subplots(figsize=(10, 6),) #dpi=1200
# ax.grid(color='grey', alpha=0.2)
ax.grid(zorder=0)

# Get number of clusters
num_clusters = len(regime_mean)

# Set width for each bar group
bar_width = 0.2

# Set positions for each cluster starting from 1
positions = np.arange(1, num_clusters + 1)

# Plot bars for each column
for i, col in enumerate(regime_mean.columns[1:]):  # Exclude "Cluster ID" column
    if col == 'slope_dicp_std':
        lbl = 'DIC'
    else:
        lbl = col.split("_")[1].upper()
    # Determine position for the bar group
    pos = positions + i * bar_width - bar_width * (len(regime_mean.columns[1:]) - 1) / 2
#     print(col)
#     print(pos)
#     print()
    
    # Plot bars
    if col == 'slope_sst_std':
        ax.barh(pos, regime_mean[col], bar_width, label=lbl, fill=False, hatch='|', zorder=3)
    if col == 'slope_dicp_std':
        ax.barh(pos, regime_mean[col], bar_width, label=lbl, fill=False, hatch='..', zorder=3)
    if col == 'slope_alk_std':
        ax.barh(pos, regime_mean[col], bar_width, label=lbl, fill=False, hatch='///', zorder=3)


# Add labels, legend, and title
ax.set_yticks(positions)
## Update the cluster labels
# ax.set_yticklabels(regime_mean.cluster)
ax.set_yticklabels(regime_names)
ax.legend()
ax.set_xlabel('Mean normalized RCs', fontsize=20)
ax.set_ylabel('Carbon uptake provinces', fontsize=20)
# ax.set_title('Mean regression coeffcients of drivers for each carbon provinces', fontsize=15)

import matplotlib as mpl
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20

# Show plot
plt.savefig(f"output_tracking/figs/cluster_details_2009_jan",
            dpi=300, bbox_inches='tight')

plt.show()